In [1]:
from IPython.core.display import display, HTML
from string import Template
import json

In [2]:
with open("movies.csv") as f:
    movies = [{"name:" : title.rstrip()} for title in f.readlines()[1:]]
with open("data.json") as f:
    matrix = json.loads("".join(f.readlines()).replace('\n', ''))
    
json.dumps(movies)
json.dumps(matrix)

'[[0, 21, 16, 14, 18, 12, 51, 17, 55, 21, 17], [21, 0, 47, 28, 37, 32, 33, 51, 39, 58, 51], [16, 47, 0, 20, 28, 29, 22, 38, 29, 31, 29], [14, 28, 20, 0, 63, 23, 26, 15, 28, 27, 23], [18, 37, 28, 63, 0, 31, 30, 17, 36, 39, 30], [12, 32, 29, 23, 31, 0, 21, 14, 28, 51, 30], [51, 33, 22, 26, 30, 21, 0, 19, 69, 32, 24], [17, 51, 38, 15, 17, 14, 19, 0, 23, 26, 17], [55, 39, 29, 28, 36, 28, 69, 23, 0, 37, 28], [21, 58, 31, 27, 39, 51, 32, 26, 37, 0, 47], [17, 51, 29, 23, 30, 30, 24, 17, 28, 47, 0]]'

In [3]:
css = '''
<style>

#graph {
  text-align: center;
}

#circle circle {
  fill: none;
  pointer-events: all;
}

.group path {
  fill-opacity: .5;
}

.title {
  font-family: Tahoma;
  font-size: 6px;
  font-weight: bold;
  text-anchor: middle;
}

path.chord {
  stroke: #000;
  stroke-width: .25px;
}

#circle:hover path.fade {
  display: none;
}

</style>
'''

js_template = Template('''
<script src="//d3js.org/d3.v3.min.js"></script>
<script>

var width = 720,
    height = 720,
    outerRadius = Math.min(width, height) / 2 - 10,
    innerRadius = outerRadius - 24;

var formatPercent = d3.format(".1%");
var arc = d3.svg.arc()
    .innerRadius(innerRadius)
    .outerRadius(outerRadius);

var layout = d3.layout.chord()
    .padding(.04)
    .sortSubgroups(d3.descending)
    .sortChords(d3.ascending);

var path = d3.svg.chord()
    .radius(innerRadius);

var svg = d3.select("#graph").append("svg")
    .attr("width", width)
    .attr("height", height)
  .append("g")
    .attr("id", "circle")
    .attr("transform", "translate(" + width / 2 + "," + height / 2 + ")");

svg.append("circle")
    .attr("r", outerRadius);


function ready(movies, matrix) {
  // Compute the chord layout.
  layout.matrix(matrix);

  // Add a group per neighborhood.
  var group = svg.selectAll(".group")
      .data(layout.groups)
    .enter().append("g")
      .attr("class", "group")
      .on("mouseover", mouseover);

  // Add a mouseover title.
  group.append("title").text(function(d, i) {
    return movies[i].name;// + ": " + formatPercent(d.value) + " of origins";
  });
  
  var colorsa = d3.scale.category20();
  var colorsb = d3.scale.category20b();  

  // Add the group arc.
  var groupPath = group.append("path")
      .attr("id", function(d, i) { return "group" + i; })
      .attr("d", arc)
      .style("fill", function(d, i) { return i >= 20 ? colorsb(i) : colorsa(i); }); //movies[i].color; });

  // Add a text label.
  var groupText = group.append("text")
      .attr("x", 6)
      .attr("dy", 15)
      .attr("class", "title");

  groupText.append("textPath")
      .attr("startOffset", "20%")
      .attr("xlink:href", function(d, i) { return "#group" + i; })
      .text(function(d, i) { return movies[i].name; });

  // Remove the labels that don't fit. :(
  groupText.filter(function(d, i) { return groupPath[0][i].getTotalLength() / 2 - 16 < this.getComputedTextLength(); })
      .remove();

  // Add the chords.
  var chord = svg.selectAll(".chord")
      .data(layout.chords)
    .enter().append("path")
      .attr("class", "chord")
      .style("fill", function(d) { return movies[d.source.index].color; })
      .attr("d", path);

  // Add an elaborate mouseover title for each chord.
  chord.append("title").text(function(d) {
    return movies[d.source.index].name
        + " -> " + movies[d.target.index].name;
  });

  function mouseover(d, i) {
    chord.classed("fade", function(p) {
      return p.source.index != i
          && p.target.index != i;
    });
  }
}

var movies = $movies;
var matrix = $matrix;

ready(movies, matrix);

</script>
''')

html_template = Template('''
<html>
$css
<body>
<div style="text-align: center">
<h2>Movie Lens Data</h2>
An overview of related movies
</div>
<div id="graph"></div>
</body>
$js
</html>
''')

js = js_template.substitute({"movies": json.dumps(movies), "matrix": json.dumps(matrix)})

HTML(html_template.substitute({"css": css, "js": js}))